In [ ]:
import GenerateHalo.py as GenH
import GenerateCluster.py as GenC
import visualization.py as vis
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt

print('This is a test notebook')